# Circulant Tensor Nuclear Norm Minimization

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def prox_2d(z, w, lmbda):
    N, T = z.shape
    temp1 = np.fft.fft2(lmbda * z - w) / lmbda
    temp2 = 1 - N * T / (lmbda * np.abs(temp1))
    temp2[temp2 <= 0] = 0
    return np.fft.ifft2(temp1 * temp2).real

def update_z(y, x, w, lmbda, eta):
    z = x + w / lmbda
    z[y != 0] = (lmbda / (lmbda + eta) * z[y != 0]
                    + eta / (lmbda + eta) * y[y != 0])
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def CTNNM(y, lmbda, maxiter = 50):
    N, T = y.shape
    eta = 100 * lmbda
    z = y.copy()
    w = y.copy()
    show_iter = 100
    for it in range(maxiter):
        x = np.fft.fft2(lmbda * z - w) / lmbda
        z = 1 - N * T / (lmbda * np.abs(x))
        z[z <= 0] = 0
        x = np.fft.ifft2(x * z).real
        z = x + w / lmbda
        z[y != 0] = ((lmbda * z + eta * y) / (lmbda + eta))[y != 0]
        w = w + lmbda * (x - z)
    return x

from ipywidgets import IntProgress
from IPython.display import display

def CTNNM_forecast(sparse_mat, pred_step, multi_step, lmbda):
    dim1, T = sparse_mat.shape
    start_time = T - pred_step
    max_count = int(np.ceil(pred_step / multi_step))
    mat_hat = np.zeros((dim1, max_count * multi_step))
    f = IntProgress(min = 0, max = max_count) # instantiate the bar
    display(f) # display the bar
    for t in range(max_count):
        if t == 0:
            mat = CTNNM(np.append(sparse_mat[:, 0 : start_time],
                                  np.zeros((dim1, multi_step)),
                                  axis = 1)[:, multi_step :],
                        lmbda)[:, - multi_step :]
        else:
            mat = CTNNM(np.append(sparse_mat[:, 0 : start_time + t * multi_step],
                                  np.zeros((dim1, multi_step)),
                                  axis = 1)[:, (t + 1) * multi_step :],
                        lmbda)[:, - multi_step :]
        mat_hat[:, t * multi_step : (t + 1) * multi_step] = mat
        del mat
        f.value = t
    small_dense_mat = sparse_mat[:, start_time : T]
    pos = np.where(small_dense_mat != 0)
    print('Prediction MAPE: {:.6}'.format(compute_mape(small_dense_mat[pos], mat_hat[pos])))
    print('Prediction RMSE: {:.6}'.format(compute_rmse(small_dense_mat[pos], mat_hat[pos])))
    print()
    return mat_hat

In [ ]:
import cupy as np
import time
import warnings
warnings.simplefilter('ignore')

dense_mat = np.load('../datasets/NYC-movement-data-set/hourly_speed_mat_2019_1.npz')['arr_0']
for month in range(2, 4):
    dense_mat = np.append(dense_mat, np.load('../datasets/NYC-movement-data-set/hourly_speed_mat_2019_{}.npz'.format(month))['arr_0'], axis = 1)
dense_mat = dense_mat[:, : 24 * 7 * 10]

pred_step = 7 * 24
N, T = dense_mat.shape
lmbda = 1e+2
week = 9
for multi_step in [1, 2, 3, 6]:
    start = time.time()
    mat_hat = CTNNM_forecast(dense_mat[:, 24 * 7 * (10 - week - 1) : 24 * 7 * 10], pred_step, multi_step, lmbda)
    print('Time horizon = {}'.format(multi_step))
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

In [ ]:
import cupy as np
import time
import warnings
warnings.simplefilter('ignore')

dense_mat = np.load('../datasets/Seattle-movement-data-set/hourly_speed_mat_2019_1.npz')['arr_0']
for month in range(2, 4):
    dense_mat = np.append(dense_mat, np.load('../datasets/Seattle-movement-data-set/hourly_speed_mat_2019_{}.npz'.format(month))['arr_0'], axis = 1)
dense_mat = dense_mat[:, : 24 * 7 * 10]

pred_step = 7 * 24
N, T = dense_mat.shape
lmbda = 1e+2
week = 9
for multi_step in [1, 2, 3, 6]:
    start = time.time()
    mat_hat = CTNNM_forecast(dense_mat[:, 24 * 7 * (10 - week - 1) : 24 * 7 * 10], pred_step, multi_step, lmbda)
    print('Time horizon = {}'.format(multi_step))
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>